# Scraping Masterarbeit --- Twitter (Versuch 1)

Eine erste Datei, um mittels der Daten von [J. Schmidt et al. (2021)](https://github.com/Leibniz-HBI/DBoeS-data-BTW21) die Twitter-Accounts der Kandidierenden zur BTW 2021 zu scrapen.

In [1]:
# basis packages
import requests
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
from pprint import pprint
import math
import json

# scrape packages: snscrape
import snscrape.modules.twitter as sntwitter

## Datensatz: Infos zu allen Kandidierenden

+ Idee: Spalte TW_user ansprechen und dann scrape-Funktion erstellen, die die komplette Datenbank durchläuft für alle, bei denen TW_user nicht 'null' ist
    + [ ] erfordert evtl.: expression, um dem TW_user-Twitternamen ein "@" vorzulagern, da von snscrape benötigt
    + [X] braucht evtl. außerdem preprocessing-Schritt, dass alle 'null'-Werte entfernt werden
+ andere Möglichkeit: manuell die Namen kopieren und eintragen

In [2]:
# datensatz als pd dataframe laden
btw21_data = pd.read_csv("./DBoeS-data-BTW21/data.csv")

# vorbereitung: entferne alle kandidierenden OHNE twitteraccount aus datenbank
btw21_rem = btw21_data[btw21_data.TW_acc != 0]

# vorbereitung 2: behalte nur die kandidierenden im datensatz, die am ende auch erfolgreich mit kandidatur waren
btw21_rem = btw21_rem[btw21_rem.MdB_2021 != 0]

# rest anzeigen --- 607 leute
btw21_rem

,ID,LAND_NR,Partei_NR,Partei_ABK,Name,Titel,Vorname,Nachname,Jahrgang,Geschlecht,...,erneut,TW_acc,TW_user,TW_id,TW_verif,FB_acc,FB_user,FB_id,FB_typ,FB_verif
0,1,5,2,SPD,"Abdi, Sanae",NaN,Sanae,Abdi,1986,1,...,0,1,_sanaeabdi,200695422,1,1,sanaeabdispd,111576226993790,1,1
1,2,8,4,FDP,"Abel, Valentin Christian",NaN,Valentin Christian,Abel,1991,2,...,0,1,Valentin_C_Abel,1699863337,1,1,valentinabel.fdp,1586699104972808,1,1
19,20,6,4,FDP,"Adler, Katja",NaN,Katja,Adler,1974,1,...,0,1,katjadler,712941191,2,1,katjadler,106967991184300,1,1
24,25,8,6,GRÜNE,"Aeffner, Stephanie",NaN,Stephanie,Aeffner,1976,1,...,0,1,s_aeffner,982288614,2,1,stephanie.aeffner,666817845,3,2
42,43,8,5,DIE LINKE,"Akbulut, Gökay",NaN,Gökay,Akbulut,1982,1,...,1,1,AkbulutGokay,800457587510476800,1,1,goekay.akbulut,1628676764050562,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165,6166,5,1,CDU,"Ziemiak, Paul",NaN,Paul,Ziemiak,1985,2,...,1,1,PaulZiemiak,288696645,1,1,paulziemiak,100044268707373,1,1
6167,6168,12,2,SPD,"Zierke, Stefan",NaN,Stefan,Zierke,1970,2,...,1,1,zierke,64220313,1,1,ZierkeSPD,234636206679771,1,1
6174,6175,6,2,SPD,"Zimmermann, Dr. Jens",Dr.,Jens,Zimmermann,1981,2,...,1,1,JensZSPD,449619355,1,1,JensZSPD,315208841871484,1,1
6188,6189,8,1,CDU,"Zippelius, Nicolas Benjamin",NaN,Nicolas Benjamin,Zippelius,1987,2,...,0,1,NZippelius,1560104953,2,1,NZippelius,102135698530662,1,1


# Test der Scraping-Funktion

Die folgenden zwei Codezellen zeigen erste Anwendungsbeispiele, wie die snscrape-Funktion für Twitter verwendet werden könnte.

## Mögliche Argumente bzw. Attribute für Tweets

Infos hierzu gezogen vom [Blogeintrag von Martin Beck](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af).

| **Attribut** | **Beschreibung** |
|:--------------|:------------------|
| *url* | URL des Tweets |
| *date* | Zeitstempel des Tweets |
| *content* (inzw. *rawContent*) | Inhalt des Tweets |
| *renderedContent* | vgl. *rawContent*, müsste aber Emojis beinhalten |
| *id* | Tweet-ID |
| *user* | userspezifische Abfragen, u. a.: username, displayname, id, description, descriptionUrls, verified, created, followersCount, friendsCount, statusesCount, favouritesCount, listedCount, mediaCount, location, protected, linkUrl, profileImageUrl, profileBannerUrl |
| *outlinks* | --- |
| *tcooutlinks* | --- |
| *replyCount* | Anzahl Antworten auf den Tweet |
| *retweetCount* | Anzahl Retweets des Tweets |
| *likeCount* | Anzahl Likes/Herzen des Tweets |
| *quoteCount* | Anzahl der Quotes des Tweets bzw. wie viele darauf geantwortet haben |
| *conversationId* | ID der gesamten Konversation (dürfte eigl. nur bei Retweets oder sowas interessant sein |
| *lang* | eine KI schätzt ab, in welcher Sprache der Tweet verfasst ist |
| *source* | von welchem Gerät der Tweet abgesetzt wurde (Computer, Smartphone, etc.) |
| *media* | Medienobjekt (Bild oder dergleichen), das an den Tweet angehängt wurde; hat eigene Unterattribute: previewUrl, fullUrl, type |
| *retweetedTweet* | wenn es ein Retweet ist, dann hier die ID des Originals |
| *quotedTweet* | wenn es ein zitierter Tweet ist, dann hier die ID des Originals |
| *mentionedUsers* | "User Objects" von Usern, die in dem Tweet direkt genannt werden |

In [49]:
# liste für tweets anlegen --- später in pd.DataFrame konvertieren
tweet_list_date = []

# scraping (mit datum: beispielsweise hier vom 01-01-2022 bis 24-11-2022)
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('from:andreasscheuer since:2020-01-01 until:2022-11-24').get_items()):
    
    # auswahl der features, die gescraped werden sollen
    tweet_list_date.append([tweet.date, tweet.id, tweet.rawContent, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.retweetedTweet, tweet.user.username])
    
# liste in pandas DataFrame umwandeln (einfachere Nutzung; bei Bedarf als CSV speicherbar)
tweets_bsp_date = pd.DataFrame(tweet_list_date, columns=['dateTime', 'tweetID', 'content', 'likes', 'replies', 'retweets', 'retweeted_yesno', 'username'])
tweets_bsp_date

,dateTime,tweetID,content,likes,replies,retweets,retweeted_yesno,username
0,2022-11-23 22:17:36+00:00,1595542104435990537,Statt tagelang über Kapitänsbinden zu diskutie...,164,241,21,None,andreasscheuer
1,2022-10-23 13:19:28+00:00,1584172657687539712,Ich bin fassungslos. Das ist ein völlig falsch...,93,107,21,None,andreasscheuer
2,2022-10-09 09:07:39+00:00,1579035853552496642,btw wir haben in meiner Amtszeit mit der @DB_B...,30,32,2,None,andreasscheuer
3,2022-10-09 09:07:38+00:00,1579035850662244352,Typisch @FlorianvonBrunn &amp; @BayernSPD! Fak...,122,80,12,None,andreasscheuer
4,2022-10-07 15:17:12+00:00,1578404077788184577,Klare Antwort auf die Frage: JA - nach den Erf...,9,11,2,None,andreasscheuer
...,...,...,...,...,...,...,...,...
246,2020-01-30 17:59:43+00:00,1222942480817098757,Heute hat der #Bundestag 2 #BMVI-Gesetzen zuge...,35,43,3,None,andreasscheuer
247,2020-01-26 22:40:01+00:00,1221563472078692352,Als Basketballer und Mensch bin ich tief betro...,78,14,5,None,andreasscheuer
248,2020-01-23 14:05:41+00:00,1220346868980428801,Ich möchte den Familien meine tiefe Anteilnahm...,71,10,6,None,andreasscheuer
249,2020-01-18 11:12:18+00:00,1218491298870648832,Unsere gemeinsame Entscheidung für unsere Sold...,107,38,6,None,andreasscheuer


In [54]:
# liste für tweets anlegen --- später in pd.DataFrame konvertieren
tweet_list_all = []

# scraping aller tweets eines politikers
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('from:andreasscheuer').get_items()):
    
    ## 1000000 tweets sind so viele, keiner der politiker sollte mehr abgesetzt haben
    if i > 1000000:
        break
    
    ## auswahl der features, die gescraped werden sollen
    tweet_list_all.append([tweet.date, tweet.id, tweet.rawContent, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.retweetedTweet, tweet.user.username])
    
# liste in pandas DataFrame umwandeln (einfachere Nutzung; bei Bedarf als CSV speicherbar)
tweets_bsp_all = pd.DataFrame(tweet_list_all, columns=['dateTime', 'tweetID', 'content', 'likes', 'replies', 'retweets', 'retweeted_yesno', 'username'])
tweets_bsp_all

,dateTime,tweetID,content,likes,replies,retweets,retweeted_yesno,username
0,2022-11-23 22:17:36+00:00,1595542104435990537,Statt tagelang über Kapitänsbinden zu diskutie...,164,242,21,None,andreasscheuer
1,2022-10-23 13:19:28+00:00,1584172657687539712,Ich bin fassungslos. Das ist ein völlig falsch...,93,107,21,None,andreasscheuer
2,2022-10-09 09:07:39+00:00,1579035853552496642,btw wir haben in meiner Amtszeit mit der @DB_B...,30,32,2,None,andreasscheuer
3,2022-10-09 09:07:38+00:00,1579035850662244352,Typisch @FlorianvonBrunn &amp; @BayernSPD! Fak...,122,80,12,None,andreasscheuer
4,2022-10-07 15:17:12+00:00,1578404077788184577,Klare Antwort auf die Frage: JA - nach den Erf...,9,11,2,None,andreasscheuer
...,...,...,...,...,...,...,...,...
1637,2009-07-07 12:41:40+00:00,2513282741,.....Und das funktioniert jetzt auch noch. Mel...,0,0,0,None,andreasscheuer
1638,2009-07-07 12:27:55+00:00,2513146242,Zum erstenmal twittere ich von meinem blackber...,14,1,4,None,andreasscheuer
1639,2009-06-30 09:52:51+00:00,2401604088,...mit unserem Regierungsprogramm für 2009-201...,0,0,0,None,andreasscheuer
1640,2009-06-26 16:02:22+00:00,2345684863,"Ich freu mich, dass ich jetzt auch auf Twitter...",0,0,0,None,andreasscheuer


# Idee: Automatisierung mittels Datensatz

Idee ist, die Spalte mit den Usernamen als Liste zu nehmen, die eine Scraping-Funktion dann einen nach dem anderen abgehen sollte.

Idee folgendermaßen umgesetzt:
1. Fragliche Spalte des Datensatzes listen lassen, Namen der gewählten MdB mit Twitteraccount erhalten und kopiert
2. Einfügen der Namen in die Scraperfunktion als zu scrapende Usernamen

Proof of Concept in kleinem Rahmen funktioniert, dauert jedoch extrem lange, wenn er wirklch alle scrapen soll. Scheint einen Request Timeout zu verursachen, derartig viele Accounts auf einmal anzufragen.

In [10]:
list(btw21_rem["TW_user"])
# das funktioniert

['_sanaeabdi',
 'Valentin_C_Abel',
 'katjadler',
 's_aeffner',
 'AkbulutGokay',
 'ReemAlabali',
 'Stephan_Albani',
 'ali_aldailami',
 'HalakMuhanad',
 'RenataAlt_MdB',
 'NAltenkamp',
 'DagmarAndres',
 'NielsAnnen',
 'arlt_johannes',
 'cad59',
 'AnAudretsch',
 'PeterAumer',
 'maik_aussendorf',
 'TBBacherle',
 'badulrichmartha',
 'ABaerbock',
 'MdB_Ulrike_Bahr',
 'baldy_daniel',
 'fbanaszak',
 'DoroBaer',
 'karlbaer',
 'NBaradari',
 'Thomas_Bareiss',
 'soerenbartol',
 'DietmarBartsch',
 'baerbelbas',
 'nicole_ae_bauer',
 'DrChristinaBaum',
 'BerndBaumannAfD',
 'LieblingXhain',
 'kathabeck',
 'RogerBeckamp',
 'jensbeeck',
 'BengtBergt',
 'MarcBernhardAfD',
 'Peter_Beyer',
 'MBiadaczMdB',
 'SteffenBilger',
 'MWBirkwald',
 'JakobBla',
 'andreasbleckmdb',
 'IngoBodtke',
 'PeterBoehringer',
 'JBrandenburgFDP',
 'BraFDP',
 'reinhardbrandl',
 'StBrandner',
 'fbrantner',
 'HBraun',
 'JuergenBraunAfD',
 'BreherSilvia',
 'Brehm_inNBGNord',
 'HeikeBrehmerMdB',
 'm_breilmann',
 'LeniBreymaier',
 'rb

In [14]:
namensliste = ['_sanaeabdi', 'Valentin_C_Abel', 'katjadler', 's_aeffner', 'AkbulutGokay',  'ReemAlabali',
 'Stephan_Albani', 'ali_aldailami', 'HalakMuhanad', 'RenataAlt_MdB', 'NAltenkamp', 'DagmarAndres', 'NielsAnnen',
 'arlt_johannes', 'cad59', 'AnAudretsch', 'PeterAumer', 'maik_aussendorf', 'TBBacherle', 'badulrichmartha',
 'ABaerbock', 'MdB_Ulrike_Bahr', 'baldy_daniel', 'fbanaszak', 'DoroBaer', 'karlbaer', 'NBaradari', 'Thomas_Bareiss',
 'soerenbartol', 'DietmarBartsch', 'baerbelbas', 'nicole_ae_bauer', 'DrChristinaBaum', 'BerndBaumannAfD',
 'LieblingXhain', 'kathabeck', 'RogerBeckamp', 'jensbeeck', 'BengtBergt', 'MarcBernhardAfD', 'Peter_Beyer',
 'MBiadaczMdB', 'SteffenBilger', 'MWBirkwald', 'JakobBla', 'andreasbleckmdb', 'IngoBodtke', 'PeterBoehringer',
 'JBrandenburgFDP', 'BraFDP', 'reinhardbrandl', 'StBrandner', 'fbrantner', 'HBraun', 'JuergenBraunAfD',
 'BreherSilvia', 'Brehm_inNBGNord', 'HeikeBrehmerMdB', 'm_breilmann', 'LeniBreymaier', 'rbrinkhaus', 'agnieszka_mdb',
 'FBsirske', 'Bubendorferfdp', 'Marcus_Buehl', 'YannickBury', 'MarcoBuschmann', 'busen_mdb', 'PetrBystronAfD',
 'isacademartori', 'larscastellucci', 'annachristmann', 'Tino_Chrupalla', 'gitta_connemann', 'juergencosse',
 'JoanaCotar', 'cjcronenberg', 'GottfriedCurio', 'MarioCzaja', 'SevimDagdelen', 'janoschdahmen', 'BernhardDaldrup',
 'VriesChristoph', 'ekindeligoez', 'HakanDemirNK', 'detzer_sandra', 'KarambaDiaby', 'MDiedenhofen', 'JanUlrichDieren',
 'tommylukoil', 'EstherDilcher', 'dittmarsabine', 'DjirSarai', 'anked', 'katdro', 'DeborahDuering', 'christianduerr',
 'Hansjoerg_Durz', 'ebner_sha', 'Leckert_', 'edelhaeusser', 'Thomas_Ehrhorn', 'sonjaeichwede', 'herr_emmerich',
 'EnglhardtKopf', 'TErndl', 'ernst_klaus', 'wiebkeesdar1', 'EskenSaskia', 'DrMEspendiller', 'MarcusFaber',
 'FahimiYasmin', 'robert_farle', 'ArianeFaescher', 'FechnerJohannes', 'UweFe', 'PeterFelser', 'SusanneFerschl',
 'emiliafester', 'fiedelseb', 'DFoest', 'FrankeEdgar', 'thorsten_frei', 'CvStetten', 'Otto_Fricke', 'MichaelFrieser',
 'Frohnmaier_AfD', 'GtzFrmming', 'max_fksr', 'funke_fabian', 'gadechens', 'schahinag', 'GansGruen', 'MatthiasGastel',
 'thomasgebhart', 'KaiGehring', 'GeisslerJonas', 'StefanGelbhaar', 'GerdesMdB', 'JN_Gesenhues', 'A_Gloeckner',
 'Hannes_Gnauck', 'NicoleGohlke', 'GoeringEckardt', 'goerke_c', 'KayGottschalk1', 'Lambsdorff', 'FabianGramling',
 'inge_graessle', 'Timon_Gremmels', 'KerstinGriese', 'groehe', 'MGrosseBroemer', 'UliGroetsch', 'GruebelMdb',
 'Erhard_Grundl', 'S_Gruetzmacher', 'SerapGueler', 'FGuentzler', 'AtesGuerpinar', 'olavgutting', 'GregorGysi',
 'ChristianHaase6', 'hacker_fdp', 'RitaHaglKehl', 'hahnflo', 'MetinHakverdi', 'M_HarderKuehnel', 'juergenhardt',
 'philipphartewig', 'sebast_hartmann', 'HarzerUlrike', 'BriHasselmann', 'MatthiasHauer', 'Jochen_Haug', 'StHeck',
 'DHeidenblut', 'PeterHeidtFDP', 'MechthildHeil', 'hubertus_heil', 'FHeiligenstadt', 'ThomasHeilmann', 'GaHeinrich',
 'linda_lobster', 'Helferich_AfD', 'KatrinHelling', 'HellmichMdB', 'KathrinAnna', 'AnkeHennigSPD', 'SusanneHennig',
 'MichaelHennric2', 'MarcHenrichmann', 'HerbrandMarkus', 'torstenherbst', 'BHerrmann_', 'NHeselhaus',
 'Martin_Hess_AfD', 'fdp_hessel', 'HierlSusanne', 'HilseMdb', 'ChristianHirte', 'ThomasHitschler', 'Nicole_Hoechst',
 'GeroHocker', 'ManuelHoeferlin', 'Hoffmann_MdB', 'bhoffmann_mdb', 'HoffmannForest', 'ToniHofreiter',
 'Leif_Erik_Holm', 'Bruno_Hoenel', 'FHoppermann', 'JasminaHostert', 'HoubenReinhard', 'HuberMdB', 'verenahubertz',
 'HumpferMarkus', 'AndrejHunko', 'olafinderbeek', 'Jacobi_AfD', 'DJanecek', 'cdu_anne', 'tj_tweets', 'GydeJ',
 'Marc_Jongen', 'c_jung77', 'Ingmar_Jung', 'jungeinberlin', 'josip_juratovic', 'avjurisch', 'KaczmarekOliver',
 'LamyaKaddor', 'Kaiser_SPD', 'KirstenKappert', 'Macit_SPD', 'AnjaKarliczek', 'CaKa_keinScheis', 'annakassautzki',
 'GabyKatzmarek', 'MalteKaufmann', 'KaufmannAfD', 'RainerKellerWES', 'MiKellner', 'RonjaKemmer', 'kersten68',
 'KatjaKeul', 'StefanKeuterAfD', 'Bina_Khan_', 'RKiesewetter', 'Kiessling_MdB', 'sven_kindler', 'katjakipping',
 'CanselK', 'helmut_kleebank', 'Ottilie_Klein', 'MdBKlein', 'VolkmarKlein', 'MariaKlSchmeink', 'NKleinwaechter',
 'larsklingbeil', 'JuliaKloeckner', 'AnniKlose', 'DanielaKluckert', 'timkluessendorf', 'axel_knoerig',
 'PascalKober', 'JensKoeppen', 'BaerbelKofler', 'koehler_fdp', 'EnricoKomning', 'AnneKoenig_', 'JoernKoenigAfD',
 'carina_konrad', 'KoobMar', 'chantal_kopf', 'CarstenKoerber', 'jankortemdb', 'KossSimona', 'SteffenKotre',
 'Dr_Rainer_Kraft', 'laura_kraft_', 'PhiKraemer', 'AnetteKramme', 'd_kreiser', 'GuntherKrichba3', 'guenterkrings',
 'Oliver_Krischer', 'MartinKroeber', 'krusehamburg', 'TKuban96', 'KonstantinKuhle', 'ChrisKuehn_mdb', 'KuehniKev',
 'RenateKuenast', 'markuskurthmdb', 'SLahrkamp', 'Ricarda_Lang', 'UlrichLange', 'ArminLaschet', 'DrLaunert',
 'Karl_Lauterbach', 'CarenLay', 'ulrichlechte', 'svenlehmann', 'LehmannSpd', 'KLeikert', 'KLHohenlohe', 'SteffiLemke',
 'DerLenzMdB', 'christian_leye', 'BodeLuiza', 'ThorstenLieb', 'EsraLimbacher', 'Helge_Limburg', 'LFLindemann',
 'helgelindh', 'AndreaLindholz', 'c_lindner', 'tobiaslindner', 'michael_g_link', 'denise_loop', 'LoetzschMdB',
 'MdB_Lucassen', 'max_lucks', 'JM_Luczak', 'DaniLudwigMdB', 'BettinaLugk', 'AnnaLuehrmann', 'OlliLuksic',
 'kristine_lutke', 'thlutze', 'HeikoMaas', 'TMachalet', 'IsabelMackensen', 'YvonneMagwas', 'HolgerMannLE',
 'TillMansmann', 'El_KaWeh_', 'ZandaMartens', 'Doro_Martin', 'ParsaMarvi', 'f_mascheck', 'KatjaMast',
 'AMattfeldt', 'zoe_gruene', 'VMayer_Lay', 'Mehli', 'TakisMehmetAli1', 'pascalmeiser', 'meister_schafft',
 'AnikoMerten', '_FriedrichMerz', 'mesarosch', 'JanMetzler', 'Meyer_FDP', 'CorinnaMiazga', 'SwantjeMichael1',
 'Kthrn_Mchl_SPD', 'IreneMihalic', 'borismijatovic', 'Amira_M_Ali', 'Conni_Moehring', 'FalkoMohrs',
 'ClaudiaMollMdB', 'SiemtjeMdB', 'Mike_Moncsek', 'MdBMonstadt', 'MMoosdorf', 'maxmordhorst', 'maxmoerseburg',
 'alexmuellerfdp', 'BetMueller', 'MdbMuller', 'GruenClaudia', 'MuellerChemnitz', 'FlorianMM', 'SaschaMuller',
 'mueller_sepp', 'smuellermdb', 'GrueneBeate', 'theliberalfrank', 'Mi_Muentefering', 'S_Muenzenmaier', 'SaraNanni',
 'rasha_nasr_', 'ZaklinNastic', 'edgar_naujok', 'Ingrid_Nestle', 'OpheliaNick', 'BrianNickholz', 'Jan_Nolte_AfD',
 'nouripour', 'JorgNurnberger', 'leoehl', 'WilfriedOellers', 'moritz_oppelt', 'josephineortleb', 'Florian_Ossner',
 'josefoster', 'vtdgsprecher', 'karo_otte', 'Gerold_Otten', 'cem_oezdemir', 'oezdemir_spd', 'oezoguz',
 'J_Pahlke', 'Dr_Pantazis', 'M_Papendieck', 'PetraPauMaHe', 'lisapaus', 'natalie_pawlik', 'JensPeick',
 'LINKEPELLI', 'victorperli', 'TobiasMPeterka', 'CPetryMdB', 'PhilippiSpd', 'PaulaPiechotta', 'StephPilsinger',
 'JanPlobner', 'christophploss', 'Pohl_MdB', 'filizgreen', 'SabinePoschmann', 'Achim_P', 'AfDProtschka',
 '_ThomasRachel', 'AlexanderRadwan', 'AloisRainer', 'docredder', 'HenningRehbaum', 'M_Reichardt_AfD',
 'MarkusReichelDD', 'HeidiReichinnek', 'AnjaReinalter', 'reinholdmdb', 'Renner_AfD', 'MartinaRenner',
 'reuther_bernd', 'YeOne_Rhie', 'b_riexinger', 'Andreas_Rimkus', 'SoenkeRix', 'DennisRohde', 'SebRoloff',
 'MartinRosemann', 'jessi_rosenthal', 'TabeaRoessner', 'MiRo_SPD', 'n_roettgen', 'ManuelaRottman', 'StefanRouenhoff',
 'TR_Bremen', 'Erwin_Rueddel', 'jethoru', 'crueffer', 'ruetzelbernd', 'RyglewskiS', 'Johann_Saathoff',
 'kassem_ts', 'th_sattelberger', 'Chr_Sauter', 'sebastian_es_', 'ingoschaeferspd', 'jamila_anna', 'f_schaeffler',
 'Herbstbernd27', 'ulle_schauws', 'NinaScheer_SPD', 'andreasscheuer', 'm_schieder', 'UdoSchiefner', 'ulschzi',
 'JanaSchimke', 'kratermensch', 'NilsSchmid', 'dieschmidt', 'jws_afd', 'Schmidt_MdB', 'schneidercar', 'Schneider_AfD',
 'PSchnieder', 'OlafScholz', 'NadineSchoen', 'lime_green_leni', 'jschraps', 'felixschreiner', 'cj_schroeder',
 'ria_schroeder', 'mischrodi', '_anja_schulz', 'Uwe_Schulz_AfD', 'K_SA', 'SvenjaSchulze68', 'FrankSchwabe',
 's_schwartze', 'Schwarz_MdB', 'ArminSchwarz', 'rischwasu', 'mseesternpauly', 'stefan_seidler', 'StSeiter',
 'Th_Seitz_AfD', 'Lina_Seitzl', 'Melis_S', 'RainerSemet', 'Martin_Sichert', 'SilberhornMdB', 'BjoernSimon',
 'Petra_Sitte_MdB', 'JudithSkudelny', 'nyke_slawik', 'TinoSorge', 'jensspahn', 'AnneSpallek', 'DirkSpaniel',
 'spellerberg_m', 'Rene_Springer', 'stadler_svenja', 'katrin_staffler', 'ninastahr', 'stamm_fibich', 'starkwatzinger',
 'till_steffen', 'StefingerMdB', 'StegemannAlbert', 'Ralf_Stegner', 'stonie_kiel', 'JoSteiniger', 'HanSteinmueller',
 'nadjasthamer', 'dieter_stier', 'gero_storjohann', 'MAStrackZi', 'bstrasser', 'W_SK', 'stuewer', 'jessica_tatti',
 'AwetTesfaiesus', 'LindaTeuteberg', 'jensteutrine', 'EUTheurer', 'michael_thews', 'StephanThomae', 'alexander_throm',
 'AntjeTillmann', 'tippelt21', 'MTodtenhausen', 'florian_toncar', 'Toens_NRW04', 'CarstenTraeger', 'JTrittin', 'derya_tn',
 'uhl_markus', 'katrinuhligbn', 'UllmannMdB', 'VolkerUllrich', 'G_UllrichFDP', 'julia_verlinden', 'johannesvogel', 'voglerk',
 'MarjaVoellers', 'EVMGreifswald', 'KonstantinNotz', 'Beatrix_vStorch', 'VoepelDirk', 'JoWadephul', 'WagenerNiklas',
 'robinwagener', 'SWagenknecht', 'yooHannes', 'Krawallstein', 'BeateWaRo', 'wanderwitz', 'ninawarken', 'sandra_weeser',
 'CarmenWegge', 'Alice_Weidel', 'DrJoeWeingarten', 'anjaweisgerber', 'SaskiaLeaRaquel', 'MariaLenaWeiss',
 'StefWenzel', 'thereallenzii', 'NicoleWestig', 'BerndWestphal4', 'h_weyel', 'Kai_Whittaker', 'AWidmannMauz', 'WolfgangWiehle',
 'KlausWiener', 'DirkWieseSPD', 'ElisabethWinke4', 'TW_EP', 'TinaWinklmann', 'ChrWirthMdB', 'Wissing', 'Janine_Wissler',
 'Witt_Uwe', 'MareikeWulf', 'JoachimWundrak', 'KayUweZiegler71', 'PaulZiemiak', 'zierke', 'JensZSPD', 'NZippelius',
 'ArmandZorn']

In [12]:
## definieren einer solchen funktion für alle tweets (1mio)
def alle(usernamen):
    # leere liste
    prod = []
    
    # scraping aller tweets eines politikers
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f'from:{usernamen}').get_items()):
    
        ## 1000000 tweets sind so viele, keiner der politiker sollte mehr abgesetzt haben
        if i > 4:
            
            break
    
        ## auswahl der features, die gescraped werden sollen
        prod.append([tweet.date, tweet.id, tweet.rawContent,
                     tweet.likeCount, tweet.replyCount, tweet.retweetCount,
                     tweet.retweetedTweet, tweet.user.username])
    
        # liste in pandas DataFrame umwandeln (einfachere Nutzung; bei Bedarf als CSV speicherbar)
        prod_end = pd.DataFrame(prod, columns=['dateTime', 'tweetID', 'content',
                                               'likes', 'replies', 'retweets',
                                               'retweeted_yesno', 'username'])
        return prod_end

In [16]:
alle("Alice_Weidel")

,dateTime,tweetID,content,likes,replies,retweets,retweeted_yesno,username
0,2022-11-23 11:23:20+00:00,1595377455367323649,"Lieber @OlafScholz, nicht nur ich, sondern Mil...",8724,704,1600,None,Alice_Weidel


# Proposal 1 from https://stackoverflow.com/questions/67939869/python-how-to-iterate-through-a-list-of-twitter-users-using-snscrape

ist ok macht aber unschöne listenobjekte

## proposal 2

inspiriert durch https://stackoverflow.com/questions/67939869/python-how-to-iterate-through-a-list-of-twitter-users-using-snscrape --- ergänzt wurde die Listenauswahl für kompletten satz aller politiker (gewählt in BT UND mit Twitteraccount)

**Anmerkung zu Code unten: testweise nur mit 3 PolitikerInnen durchgeführt, die (NACH MEINER EIGENEN ANNAHME) wahrscheinlich sehr viel tweeten. Hier dauert für Zeitraum Jan. 2020 bis heute (24.11.2022) die Abfrage gut 5 Minuten. Nimmt man noch einen vierten Politiker (hier Scheuer) hinzu, so dauert es noch länger --- jedoch wenigstens mit Ergebnis und kein Server-Timeout.**

In [24]:
#date_range = []

namensliste_kurz = ["Alice_Weidel", "abaerbock", "OlafScholz", "andreasscheuer", '_sanaeabdi', 'Valentin_C_Abel', 'katjadler', 's_aeffner', 'AkbulutGokay',  'ReemAlabali',
 'Stephan_Albani', 'ali_aldailami', 'HalakMuhanad', 'RenataAlt_MdB', 'NAltenkamp']

prod1 = []

for n, k in enumerate(namensliste_kurz):
    
    for i,tweet in enumerate(sntwitter
                             .TwitterSearchScraper('from:{} since:2020-01-01 until:2022-11-24'
                                                   .format(namensliste_kurz[n]))
                             .get_items()):
        
        if i > 10000: ## stopp-bedingung für irrationalen fall dass jemand mehr als 10.000 tweets in der zeit macht
        
            break
        
        ## auswahl der features, die gescraped werden sollen
        prod1.append([tweet.date, tweet.id, tweet.rawContent,
                     tweet.likeCount, tweet.replyCount, tweet.retweetCount,
                     tweet.retweetedTweet, tweet.user.username])
    
        # liste in pandas DataFrame umwandeln (einfachere Nutzung; bei Bedarf als CSV speicherbar)
        prod1_end = pd.DataFrame(prod1, columns=['dateTime', 'tweetID', 'content',
                                               'likes', 'replies', 'retweets',
                                               'retweeted_yesno', 'username'])
        
prod1_end

,dateTime,tweetID,content,likes,replies,retweets,retweeted_yesno,username
0,2022-11-23 11:23:20+00:00,1595377455367323649,"Lieber @OlafScholz, nicht nur ich, sondern Mil...",8753,706,1604,None,Alice_Weidel
1,2022-11-22 08:49:38+00:00,1594976386292260864,"#Baerbock: ""Wir lassen #Moldau weder in Kälte ...",8474,1203,1757,None,Alice_Weidel
2,2022-11-20 19:19:31+00:00,1594410126685122560,"""Wir werden alles dafür tun, dass nicht nur #D...",6676,792,934,None,Alice_Weidel
3,2022-11-20 18:48:54+00:00,1594402421224411136,"Der Wunsch nach einer #Klimaschutz-Politik, di...",4414,340,835,None,Alice_Weidel
4,2022-11-18 14:31:00+00:00,1593612743243874304,Das heroische #G20-Bild mit #Scholz im Kreise ...,6224,543,1094,None,Alice_Weidel
...,...,...,...,...,...,...,...,...
7703,2020-01-13 13:19:59+00:00,1216711488875978752,Täglich grüßt das Murmeltier: Wolfgang #Schäub...,15,2,4,None,RenataAlt_MdB
7704,2020-01-09 09:31:08+00:00,1215204347635736577,.@HeikoMaas &amp; #EU müssen die Chance ergrei...,8,0,2,None,RenataAlt_MdB
7705,2020-01-08 12:22:38+00:00,1214885117472231424,Reise #Putin|s in die #Türkei zeigt Bedeutung ...,1,1,0,None,RenataAlt_MdB
7706,2020-01-07 09:42:55+00:00,1214482537256488961,Verlegung der #Bundeswehr aus dem #Irak ist ri...,1,0,0,None,RenataAlt_MdB
